In [251]:
%matplotlib widget
import pandas as pd
import numpy as np
from scipy.signal import find_peaks
import matplotlib.pyplot as plt 
from matplotlib.ticker import PercentFormatter
import pickle
secinday = 24*3600

def find_nearest(array, value):
    """Finds the nearest value in an array and outputs a index.
    This function was found in 
    https://stackoverflow.com/questions/2566412/find-nearest-value-in-numpy-array

    Parameters:
    -----------
    array: array to be looked into
    value: single value to look for into the array

    Output:
    -------
    index of the closest value in the array
    """
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

In [267]:
#Import meltwater input calculated with meltmodel for JEME
meltmodel = pd.read_csv('Field_Data/JEME_QSUH.csv', index_col=0, parse_dates=True)
meltmodel = meltmodel.dropna()
meltmodel = meltmodel['2017/05/30':'2017/08/17 12:00']#['2017/07/19':'2017/09/01']#['2017/05/30':'2017/09/01']


#Import head measurement for JEME
jeme_moulin = pd.read_csv('Field_Data/head_jeme.csv', index_col=1)
jeme_moulin = jeme_moulin.dropna()
jeme_moulin = jeme_moulin[:228*secinday]
#h_real = jeme_moulin.head_bed.to_numpy()
#h_soy = jeme_moulin.soy.to_numpy()
#h_doy = h_soy/secinday



picklefile = open('Pickles_TC/bf0_fix_TC_nobound', 'rb')
bf0_fix = pickle.load(picklefile)
picklefile.close()


#[1.32749e7:1.56025e7]

picklefile = open('Pickles_TC/bf2_fix_TC_nobound', 'rb')
bf2_fix = pickle.load(picklefile)
picklefile.close()

start = find_nearest(bf0_fix.dict['time'], 1.32749e7)
end = find_nearest(bf0_fix.dict['time'], 1.4568e7)#1.56025e7)

head_bf0 = np.array(bf0_fix.dict['head'][start:end])
time_bf0 = np.array(bf0_fix.dict['time'][start:end])

head_bf2 = np.array(bf2_fix.dict['head'][start:end])
time_bf2 = np.array(bf2_fix.dict['time'][start:end])

In [268]:
head_bf2

array([317.97002869, 319.43876732, 320.90016159, ..., 306.62167004,
       307.78138515, 308.93436724])

In [269]:
plt.figure()
plt.plot(time_bf0, head_bf0)
plt.plot(time_bf2, head_bf2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [119]:

Qmod_peaks = find_peaks(meltmodel.Q, distance = 20, height=0)# height = 0.015)
max_height = Qmod_peaks[1]['peak_heights']
max_pos = meltmodel.index[Qmod_peaks[0]] 

Qmod_minima = find_peaks(meltmodel.Q*-1, distance = 18,) #height = -0.9)
min_pos = meltmodel.index[Qmod_minima[0]]  
min_height = meltmodel.Q[Qmod_minima[0]]#*-1


plt.figure()
plt.plot(meltmodel.index,meltmodel.Q)
plt.plot(max_pos, max_height,'o')#, color = 'r', s = 10, marker = 'D', label = 'maxima')
plt.plot(min_pos, min_height,'o')#, color = 'r', s = 10, marker = 'D', label = 'maxima')

Ain = max_height-min_height
Qin_mean = (max_height+min_height)/2
Ain_star = Ain/Qin_mean


Ain_star_average = np.zeros(len(Ain)-5)

for i in np.arange(len(Ain)-5):
    Ain_star_average[i] = np.mean([Ain_star[i],Ain_star[i+1],Ain_star[i+2],Ain_star[i+3],Ain_star[i+4]])

std = np.std(Ain_star_average)
mean = np.mean(Ain_star_average)
print('std=',std)
print('mean=',mean)

std_tot = np.std(Ain_star)
mean_tot = np.mean(Ain_star)
print('std_tot=',std_tot)
print('mean_tot=',mean_tot)

plt.figure()
plt.hist(Ain_star_average,weights=np.ones(len(Ain_star_average)) / len(Ain_star_average))#, bins=10)
plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
plt.xlabel('$A_{in}*$')
plt.title('mean')



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [220]:


Qmod_peaks = find_peaks(jeme_moulin.head_bed, distance = 22*4, height=0)# height = 0.015)
max_height = Qmod_peaks[1]['peak_heights']
max_pos = jeme_moulin.index[Qmod_peaks[0]] 

Qmod_minima = find_peaks(jeme_moulin.head_bed*-1, distance = 20*4)
min_height = jeme_moulin.head_bed.to_numpy()[Qmod_minima[0]]#*-1
min_pos = jeme_moulin.index[Qmod_minima[0]] 

max_height = np.delete(max_height,[3,4,5,8,9])
max_pos = np.delete(max_pos,[3,4,5,8,9])
min_height = np.delete(min_height,[0,4,5,6,7,10,11])
min_pos = np.delete(min_pos,[0,4,5,6,7,10,11])

plt.figure()
plt.plot(jeme_moulin.index,jeme_moulin.head_bed)
plt.plot(max_pos,max_height,'o')#, color = 'r', s = 10, marker = 'D', label = 'maxima')
plt.plot(min_pos,min_height,'o')#, color = 'r', s = 10, marker = 'D', label = 'maxima')

print(len(max_height))
print(len(min_height))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

23
23


In [259]:
jeme_moulin.index

Float64Index([17356500.0, 17357400.0, 17358300.0, 17359200.0, 17360100.0,
              17361000.0, 17361900.0, 17362800.0, 17363700.0, 17364600.0,
              ...
              19691100.0, 19692000.0, 19692900.0, 19693800.0, 19694700.0,
              19695600.0, 19696500.0, 19697400.0, 19698300.0, 19699200.0],
             dtype='float64', name='soy', length=2959)

In [231]:


Qmod_peaks = find_peaks(jeme_moulin.head_bed, distance = 22*4, height=0)# height = 0.015)
max_height = Qmod_peaks[1]['peak_heights']
max_pos = jeme_moulin.index[Qmod_peaks[0]] 

Qmod_minima = find_peaks(jeme_moulin.head_bed*-1, distance = 20*4)
min_height = jeme_moulin.head_bed.to_numpy()[Qmod_minima[0]]#*-1
min_pos = jeme_moulin.index[Qmod_minima[0]] 

max_height = np.delete(max_height,[3,4,5,8,9])
max_pos = np.delete(max_pos,[3,4,5,8,9])
min_height = np.delete(min_height,[0,4,5,6,7,10,11])
min_pos = np.delete(min_pos,[0,4,5,6,7,10,11])

plt.figure()
plt.plot(jeme_moulin.index,jeme_moulin.head_bed)
plt.plot(max_pos,max_height,'o')#, color = 'r', s = 10, marker = 'D', label = 'maxima')
plt.plot(min_pos,min_height,'o')#, color = 'r', s = 10, marker = 'D', label = 'maxima')

print(len(max_height))
print(len(min_height))

Ain = max_height-min_height
Qin_mean = (max_height+min_height)/2
Ain_star = Ain/500


Ain_star_average = np.zeros(len(Ain)-5)

for i in np.arange(len(Ain)-5):
    Ain_star_average[i] = np.mean([Ain_star[i],Ain_star[i+1],Ain_star[i+2],Ain_star[i+3],Ain_star[i+4]])

std = np.std(Ain_star_average)
mean = np.mean(Ain_star_average)
print('std=',std)
print('mean=',mean)

std_tot = np.std(Ain_star)
mean_tot = np.mean(Ain_star)
print('std_tot=',std_tot)
print('mean_tot=',mean_tot)

plt.figure()
plt.hist(Ain_star_average,weights=np.ones(len(Ain_star_average)) / len(Ain_star_average))#, bins=10)
plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
plt.xlabel('$A_{in}*$')
plt.title('mean')



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

23
23
std= 0.02794017809213546
mean= 0.09749195555555555
std_tot= 0.039536517845623134
mean_tot= 0.09816834782608695


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'mean')

In [271]:
Qmod_peaks = find_peaks(head_bf0, distance = 22*4, height=0)# height = 0.015)
max_height = Qmod_peaks[1]['peak_heights']
max_pos = time_bf0[Qmod_peaks[0]] 

Qmod_minima = find_peaks(head_bf0*-1, distance = 20*4)
min_height = head_bf0[Qmod_minima[0]]#*-1
min_pos = time_bf0[Qmod_minima[0]] 


plt.figure()
plt.plot(time_bf0,head_bf0)
plt.plot(max_pos,max_height,'o')#, color = 'r', s = 10, marker = 'D', label = 'maxima')
plt.plot(min_pos,min_height,'o')#, color = 'r', s = 10, marker = 'D', label = 'maxima')

print(len(max_height))
print(len(min_height))

Ain = max_height-min_height
Qin_mean = (max_height+min_height)/2
Ain_star = Ain/500


Ain_star_average = np.zeros(len(Ain)-5)

for i in np.arange(len(Ain)-5):
    Ain_star_average[i] = np.mean([Ain_star[i],Ain_star[i+1],Ain_star[i+2],Ain_star[i+3],Ain_star[i+4]])

std = np.std(Ain_star_average)
mean = np.mean(Ain_star_average)
print('std=',std)
print('mean=',mean)

std_tot = np.std(Ain_star)
mean_tot = np.mean(Ain_star)
print('std_tot=',std_tot)
print('mean_tot=',mean_tot)

plt.figure()
plt.hist(Ain_star,weights=np.ones(len(Ain_star)) / len(Ain_star))#, bins=10)
plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
plt.xlabel('$A_{in}*$')
plt.title('mean')




Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

15
15
std= 0.23094161070331826
mean= 0.6463800179964244
std_tot= 0.3648099247260045
mean_tot= 0.7530129979548082


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'mean')

In [266]:
Qmod_peaks = find_peaks(head_bf2, distance = 22*4, height=0)# height = 0.015)
max_height = Qmod_peaks[1]['peak_heights']
max_pos = time_bf2[Qmod_peaks[0]] 

Qmod_minima = find_peaks(head_bf2*-1, distance = 20*4)
min_height = head_bf2[Qmod_minima[0]]#*-1
min_pos = time_bf2[Qmod_minima[0]] 


plt.figure()
plt.plot(time_bf2,head_bf2)
plt.plot(max_pos,max_height,'o')#, color = 'r', s = 10, marker = 'D', label = 'maxima')
plt.plot(min_pos,min_height,'o')#, color = 'r', s = 10, marker = 'D', label = 'maxima')

print(len(max_height))
print(len(min_height))

Ain = max_height-min_height
Qin_mean = (max_height+min_height)/2
Ain_star = Ain/500


Ain_star_average = np.zeros(len(Ain)-5)

for i in np.arange(len(Ain)-5):
    Ain_star_average[i] = np.mean([Ain_star[i],Ain_star[i+1],Ain_star[i+2],Ain_star[i+3],Ain_star[i+4]])

std = np.std(Ain_star_average)
mean = np.mean(Ain_star_average)
print('std=',std)
print('mean=',mean)

std_tot = np.std(Ain_star)
mean_tot = np.mean(Ain_star)
print('std_tot=',std_tot)
print('mean_tot=',mean_tot)

plt.figure()
plt.hist(Ain_star_average,weights=np.ones(len(Ain_star_average)) / len(Ain_star_average))#, bins=10)
plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
plt.xlabel('$A_{in}*$')
plt.title('mean')



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

27
27
std= 0.00885915761825675
mean= 0.08540035080539611
std_tot= 0.016423559209679018
mean_tot= 0.08691129778608468


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'mean')